In [1]:
import requests

url = 'https://apewisdom.io/api/v1.0/filter/all-stocks'

try:
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()
    print("Response JSON:")
    print(data)

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Response JSON:
{'count': 1111, 'pages': 12, 'current_page': 1, 'results': [{'rank': 1, 'ticker': 'SPY', 'name': 'SPDR S&amp;P 500 ETF Trust', 'mentions': 831, 'upvotes': 5214, 'rank_24h_ago': 1, 'mentions_24h_ago': 667}, {'rank': 2, 'ticker': 'TSLA', 'name': 'Tesla', 'mentions': 385, 'upvotes': 3189, 'rank_24h_ago': 2, 'mentions_24h_ago': 404}, {'rank': 3, 'ticker': 'SMCI', 'name': 'Supermicro', 'mentions': 316, 'upvotes': 1778, 'rank_24h_ago': 60, 'mentions_24h_ago': 11}, {'rank': 4, 'ticker': 'NVDA', 'name': 'NVIDIA', 'mentions': 293, 'upvotes': 1522, 'rank_24h_ago': 3, 'mentions_24h_ago': 164}, {'rank': 5, 'ticker': 'META', 'name': 'Meta Platforms (Facebook)', 'mentions': 212, 'upvotes': 776, 'rank_24h_ago': 25, 'mentions_24h_ago': 39}, {'rank': 6, 'ticker': 'HOOD', 'name': 'Robinhood', 'mentions': 182, 'upvotes': 530, 'rank_24h_ago': 22, 'mentions_24h_ago': 47}, {'rank': 7, 'ticker': 'MSFT', 'name': 'Microsoft', 'mentions': 180, 'upvotes': 597, 'rank_24h_ago': 40, 'mentions_24h_ago

In [2]:
import pandas as pd
df = pd.DataFrame(data['results'])
df.head()

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago
0,1,SPY,SPDR S&amp;P 500 ETF Trust,831,5214,1,667.0
1,2,TSLA,Tesla,385,3189,2,404.0
2,3,SMCI,Supermicro,316,1778,60,11.0
3,4,NVDA,NVIDIA,293,1522,3,164.0
4,5,META,Meta Platforms (Facebook),212,776,25,39.0


In [3]:
df[['ticker']]

,ticker
0,SPY
1,TSLA
2,SMCI
3,NVDA
4,META
...,...
95,TDS
96,CGC
97,PUMP
98,GEO


In [4]:
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import re
def get_stock_details(ticker_symbol):

    url = f'https://apewisdom.io/stocks/{ticker_symbol}/'
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')
    details_div = soup.find('div', class_='row', style='font-size: 18px; margin-top: 20px; margin-bottom: 10px;')

    if details_div:
        details_text = details_div.get_text(separator="\n").strip()
    else:
        details_text = f"Details div not found for {ticker_symbol}."

    match = re.search(r'keywords: (.*)', details_text)
    if match:
        keywords = match.group(1).split(",")
        keywords = [keyword.strip().replace('"', '') for keyword in keywords]
    else:
        keywords = []
    stock_data = yf.Ticker(ticker_symbol)
    try:
        hist = stock_data.history(period="1d")
        if not hist.empty:
            current_price = hist['Close'].iloc[-1]
        else:
            current_price = 'Price not found'
    except Exception as e:
        current_price = f'Error fetching price: {e}'

    positive_percentage = None
    match = re.search(r'(\d+)% positive comments', details_text)
    if match:
        positive_percentage = int(match.group(1))

    return details_text, current_price, positive_percentage, keywords

In [5]:
get_stock_details('AAPL')

('During the last 24H AAPL (Apple) was mentioned 46 times on WallstreetBets by a total of 35 different users.\nApple received 57% positive comments versus 43% negative comments.\nMost comonly the AAPL ticker was associated with the keywords: "earnings","amzn","green","puts","msft"',
 212.5,
 57,
 ['earnings', 'amzn', 'green', 'puts', 'msft'])

In [6]:
get_stock_details('MSFT')

('During the last 24H MSFT (Microsoft) was mentioned 162 times on WallstreetBets by a total of 121 different users.\nMicrosoft received 63% positive comments versus 37% negative comments.\nMost comonly the MSFT ticker was associated with the keywords: "meta","earnings","calls","img","up"',
 395.260009765625,
 63,
 ['meta', 'earnings', 'calls', 'img', 'up'])

In [7]:
get_stock_details('TSLA')

('During the last 24H TSLA (Tesla) was mentioned 356 times on WallstreetBets by a total of 225 different users.\nTesla received 50% positive comments versus 50% negative comments.\nMost comonly the TSLA ticker was associated with the keywords: "bet","before","puts","up","img"',
 282.1600036621094,
 50,
 ['bet', 'before', 'puts', 'up', 'img'])

In [8]:
get_stock_details('GOOGL')

('During the last 24H GOOGL (Alphabet (Google)) was mentioned 22 times on WallstreetBets by a total of 18 different users.\nAlphabet (Google) received 69% positive comments versus 31% negative comments.\nMost comonly the GOOGL ticker was associated with the keywords: "meta","amzn","msft","calls","good"',
 158.8000030517578,
 69,
 ['meta', 'amzn', 'msft', 'calls', 'good'])

In [9]:
get_stock_details('SPY')

('During the last 24H SPY (SPDR S&P 500 ETF Trust\r\n\r\n) was mentioned 779 times on WallstreetBets by a total of 460 different users.\nSPDR S&P 500 ETF Trust\r\n\r\n received 56% positive comments versus 44% negative comments.\nMost comonly the SPY ticker was associated with the keywords: "bet","before","img","goes","puts"',
 554.5399780273438,
 56,
 ['bet', 'before', 'img', 'goes', 'puts'])

In [10]:
# for ticker in df['ticker']:
#     print(ticker)
#     print('-----------------------------')
#     # print(get_stock_details(ticker))
#     details, price, positive_pct = get_stock_details(ticker)
#     print(f"\nCurrent stock price: {price}")
#     print(f"Positive Sentiment: {positive_pct}%")
#     print(f"\nSentiment details:\n{details}")
#     print('-----------------------------')

In [11]:
def fetch_details(ticker):
    details, price, positive_pct, keywords = get_stock_details(ticker)
    return pd.Series({
        'current_price': price,
        'positive_sentiment_pct': positive_pct,
        'sentiment_details': details,
        'keywords': keywords
    })

In [12]:
df[['current_price', 'positive_sentiment_pct', 'sentiment_details', 'keywords']] = df['ticker'].apply(fetch_details)

$BIG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RE: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$ME: possibly delisted; no price data found  (period=1d)


In [15]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f'stock_data_{timestamp}.csv'
df.to_csv(filename, index=False)

In [13]:
df[df['mentions']>=10]

,rank,ticker,name,mentions,upvotes,rank_24h_ago,mentions_24h_ago,current_price,positive_sentiment_pct,sentiment_details,keywords
0,1,SPY,SPDR S&amp;P 500 ETF Trust,831,5214,1,667.0,554.539978,56.0,During the last 24H SPY (SPDR S&P 500 ETF Trus...,"[bet, before, img, goes, puts]"
1,2,TSLA,Tesla,385,3189,2,404.0,282.160004,50.0,During the last 24H TSLA (Tesla) was mentioned...,"[bet, before, puts, up, img]"
2,3,SMCI,Supermicro,316,1778,60,11.0,31.860001,50.0,During the last 24H SMCI (Supermicro) was ment...,"[img, emote, earnings, th, nvda]"
3,4,NVDA,NVIDIA,293,1522,3,164.0,108.919998,51.0,During the last 24H NVDA (NVIDIA) was mentione...,"[smci, img, bet, emote, calls]"
4,5,META,Meta Platforms (Facebook),212,776,25,39.0,549.0,61.0,During the last 24H META (Meta Platforms (Face...,"[msft, earnings, calls, img, hood]"
...,...,...,...,...,...,...,...,...,...,...,...
87,88,SMH,VanEck Vectors ETF Trust - VanEck Vectors Semi...,10,21,112,4.0,211.279999,100.0,During the last 24H SMH (VanEck Vectors ETF Tr...,"[mstr, exposure, correlated, title, year]"
88,89,GNLN,Greenlane,10,25,82,10.0,0.0113,67.0,During the last 24H GNLN (Greenlane) was menti...,[]
89,90,NX,Quanex Building Products,10,2211,843,1.0,16.440001,29.0,During the last 24H NX (Quanex Building Produc...,"[earnings, imports, tariffs, drop, down]"
90,91,DC,Dakota Gold,10,26,54,14.0,2.7,100.0,During the last 24H DC (Dakota Gold) was menti...,"[fox, houston, washington, youtube, lavishly]"
